# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




In [1]:
%%capture
!pip install dlt[duckdb]

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [3]:
sum = sum([n**0.5 for n in range(1,6)])
print(sum)

8.382332347441762


In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**

In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [7]:
people_1()

<generator object people_1 at 0x7f36ff15fc30>

## 2.1 Load the first generator and calculate sum of ages

In [37]:
import dlt

pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

load_info = pipeline.run(people_1(), 
                         table_name="peoples", 
                         write_disposition="replace")

print(load_info)

Pipeline dlt_ipykernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/rustam/Learning/data-engineering-zoomcamp/workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707564522.7428331 is LOADED and contains no failed jobs


In [38]:
# show outcome

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

In [39]:
# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ peoples             │
└─────────────────────┘

In [40]:
# and the data
print("table:")

peoples = conn.sql("SELECT * FROM peoples").df()
display(peoples)

table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707564522.7428331,pwoPkPt5WVWzgg,None
1,2,Person_2,27,City_A,1707564522.7428331,R1MoGkaAy2RukA,None
2,3,Person_3,28,City_A,1707564522.7428331,uxAPcCNO68+GOQ,None
3,4,Person_4,29,City_A,1707564522.7428331,To7qv2VEyLPWKw,None
4,5,Person_5,30,City_A,1707564522.7428331,BX6fAwdbrOUPHA,None


In [41]:
sum_of_ages = conn.sql("select sum(age) from peoples")
display(sum_of_ages)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

## 2.2 Append the second generator to the same table as the first

In [42]:
load_info = pipeline.run(people_2(), 
                         table_name="peoples", 
                         write_disposition="append")

print(load_info)

Pipeline dlt_ipykernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/rustam/Learning/data-engineering-zoomcamp/workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707564532.954019 is LOADED and contains no failed jobs


In [43]:
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n table:")

peoples = conn.sql("SELECT * FROM peoples").df()
display(peoples)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ peoples             │
└─────────────────────┘




 table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707564522.7428331,pwoPkPt5WVWzgg,None
1,2,Person_2,27,City_A,1707564522.7428331,R1MoGkaAy2RukA,None
2,3,Person_3,28,City_A,1707564522.7428331,uxAPcCNO68+GOQ,None
3,4,Person_4,29,City_A,1707564522.7428331,To7qv2VEyLPWKw,None
4,5,Person_5,30,City_A,1707564522.7428331,BX6fAwdbrOUPHA,None
5,3,Person_3,33,City_B,1707564532.954019,+n9M04IyAFUdow,Job_3
6,4,Person_4,34,City_B,1707564532.954019,alCjl2+GU+aLug,Job_4
7,5,Person_5,35,City_B,1707564532.954019,CIjUe+PlR/W6Lg,Job_5
8,6,Person_6,36,City_B,1707564532.954019,gFLlG2GAzAO60w,Job_6
9,7,Person_7,37,City_B,1707564532.954019,zOBzZaOxvYco/Q,Job_7


In [44]:
sum_of_ages = conn.sql("select sum(age) from peoples")
display(sum_of_ages)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


It seems we don't need to append but to merge data. Let's delete newly inserted data

In [45]:
peoples = conn.sql("SELECT * FROM peoples where occupation is null").df()
display(peoples)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707564522.7428331,pwoPkPt5WVWzgg,None
1,2,Person_2,27,City_A,1707564522.7428331,R1MoGkaAy2RukA,None
2,3,Person_3,28,City_A,1707564522.7428331,uxAPcCNO68+GOQ,None
3,4,Person_4,29,City_A,1707564522.7428331,To7qv2VEyLPWKw,None
4,5,Person_5,30,City_A,1707564522.7428331,BX6fAwdbrOUPHA,None


In [46]:
load_info = pipeline.run(people_2(), 
                         table_name="peoples", 
                         primary_key="id",
                         write_disposition="merge")

print(load_info)

Pipeline dlt_ipykernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/rustam/Learning/data-engineering-zoomcamp/workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707564565.5193353 is LOADED and contains no failed jobs


In [47]:
peoples = conn.sql("SELECT * FROM peoples").df()
display(peoples)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707564522.7428331,pwoPkPt5WVWzgg,None
1,2,Person_2,27,City_A,1707564522.7428331,R1MoGkaAy2RukA,None
2,7,Person_7,37,City_B,1707564565.5193353,ErB2QGyqTIIc1Q,Job_7
3,5,Person_5,35,City_B,1707564565.5193353,bZGLI10QgrVLwQ,Job_5
4,8,Person_8,38,City_B,1707564565.5193353,jlmLphA4y7ZyEQ,Job_8
5,4,Person_4,34,City_B,1707564565.5193353,48q5S6ohKBmyig,Job_4
6,3,Person_3,33,City_B,1707564565.5193353,OJNcfXFo6fcErw,Job_3
7,6,Person_6,36,City_B,1707564565.5193353,l2svl5Vre8NC0g,Job_6


### calculate the sum of all ages of people

In [48]:
sum_of_ages = conn.sql("select sum(age) from peoples")
display(sum_of_ages)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX